In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
opt = SolverFactory('glpk').executable = 'D:\download\glpk-4.65\w64'

In [52]:
#Read generators and demand data as two dataframes  Desktop717data
df_genData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="GeneratorsData",skiprows=1) #read data from generators tab and skip the first row
df_AverloadsData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="AverageloadData",skiprows=1)
df_PeakloadsData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="PeakLoadData",skiprows=1) 
df_linesData=pd.read_excel("Desktop/717/data/A11.xlsx",sheet_name="LinesData",skiprows=1) 

In [53]:
#define the size of the sets
NumGens=len(df_genData)
NumNodes=len(df_AverloadsData)
NumLines=len(df_linesData)
#T=len(df_demandData)
print("We read data for", NumGens, "power generating units")
print("We read data for", NumNodes, "nodes")
print("We read data for", NumLines, "transmission lines")

We read data for 28 power generating units
We read data for 3 nodes
We read data for 3 transmission lines


In [54]:
#Define Indexes
G=np.array([g for g in range(0,NumGens)])  
N=np.array([n for n in range(0,NumNodes)])
L=np.array([l for l in range(0,NumLines)])

In [55]:
df_genData.columns

Index(['GenName', 'Node', 'FixedCost', 'SDCost', 'SUCost', 'VarCost', 'MaxGen',
       'SpringMaxGen', 'MinGen', 'RampDown', 'RampSD', 'RampSU', 'RampUp',
       'MinUpTime', 'MinDownTime', 'ReqUp', 'ReqDown', 'InitialStatus',
       'InitialGen', 'Plant Name', 'Fuel Type', 'Average Emission Rate'],
      dtype='object')

In [56]:
##Declare the generator's parameters and assign them the data we read before
VarCost=df_genData.loc[:,'VarCost'].to_numpy()
PMax=df_genData.loc[:,'MaxGen'].to_numpy()
SpringPMax=df_genData.loc[:,'SpringMaxGen'].to_numpy()
PMin=df_genData.loc[:,'MinGen'].to_numpy()
GenNode=df_genData.loc[:,'Node'].to_numpy()

In [7]:
df_linesData.columns  #In the data we defined 3 lines as: L1-2, L2-3, and L3-1.
#We could have defined them in other way, but this is convenient for the KVL equation

Index(['LineName', 'NodeFrom', 'NodeTo', 'Reactance', 'Capacity'], dtype='object')

In [57]:
##Declare the transmission lines' parameters and assign them the data we read before
NodeFrom=df_linesData.loc[:,'NodeFrom'].to_numpy()
NodeTo=df_linesData.loc[:,'NodeTo'].to_numpy()
LineReactance=df_linesData.loc[:,'Reactance'].to_numpy()
LineCapacity=df_linesData.loc[:,'Capacity'].to_numpy()

In [9]:
df_AverloadsData.columns

Index(['LoadName', 'Node', 'AWinterPeak', 'AWinterOffPeak', 'ASpringPeak',
       'ASpringOffPeak', 'ASummerPeak', 'ASummerOffPeak', 'AFallPeak',
       'AFallOffPeak'],
      dtype='object')

In [58]:
ALoadBus=df_AverloadsData.loc[:,'Node'].to_numpy( )
APeakWinterDemand=df_AverloadsData.loc[:,'AWinterPeak'].to_numpy( )
AOffPeakWinterDemand=df_AverloadsData.loc[:,'AWinterOffPeak'].to_numpy( )
APeakSummerDemand=df_AverloadsData.loc[:,'ASummerPeak'].to_numpy( )
AOffPeakSummerDemand=df_AverloadsData.loc[:,'ASummerOffPeak'].to_numpy( )
APeakFallDemand=df_AverloadsData.loc[:,'AFallPeak'].to_numpy( )
AOffPeakFallDemand=df_AverloadsData.loc[:,'AFallOffPeak'].to_numpy( )
APeakSpringDemand=df_AverloadsData.loc[:,'ASpringPeak'].to_numpy( )
AOffPeakSpringDemand=df_AverloadsData.loc[:,'ASpringOffPeak'].to_numpy( )

In [11]:
AOffPeakWinterDemand

array([3300.8, 7426.8, 5776.4])

In [12]:
# define a function indicatorMatrix that creates indicator matrices to relate generators with their node
# and relate lines with the nodes
#dataRows is a column vector of NumRows rows. Each element says the column where this row should be a one in the final matrix
def IndicatorMatrix(NumRows, NumCols, dataRows):
    matrix= np.zeros((NumRows, NumCols),dtype=int)
    for i in range(0,NumRows):
        matrix[i,dataRows[i]-1]=1
    return matrix

In [13]:
#Create indicator matrices using the function defined above
#GeneratorInBus is a matrix of G rows and N columns that has 1 in the position g,n if generator g is AT node N, 0 otherwise
GeneratorInBus=IndicatorMatrix(NumGens,NumNodes,GenNode)
#LineFromBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l departs FROM node N, 0 otherwise
LineFromBus=IndicatorMatrix(NumLines,NumNodes,NodeFrom)
#LineToBus is an indicator matrix of L rows and N columns that has 1 in the position l,n if line l arrives TO node N, 0 otherwise
LineToBus=IndicatorMatrix(NumLines,NumNodes,NodeTo)

In [14]:
GeneratorInBus

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [29]:
#UC MODEL of Average winter peak hour
def APeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)
                                            -sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G)
                                            == APeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m



In [16]:
AOffPeakWinterDemand

array([3300.8, 7426.8, 5776.4])

In [32]:
#UC MODEL of Average winter offpeak hour
def AOffPeakWinterED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == AOffPeakWinterDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m


In [30]:
m=APeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 349856.604
  Upper bound: 349856.604
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.056145668029785156
# ----------------------------------------------------------
#   Solution Information
# -----------------------

In [31]:
print('SOLUTION-Average winter peak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average winter peak
The total system cost is = $ 349856.60400000005
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 0.00 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 0.00 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 0.00 
generator 19 = 569.55 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 384.23 
line  2 = -4.97 
line  3 = -379.27 
LMPs in $/MWh
Node  1 = 31.00
Node  2 = 31.00
Node  3 = 31.00


In [33]:
m= AOffPeakWinterED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 268630.747
  Upper bound: 268630.747
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06637144088745117
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [34]:
print('SOLUTION-Average winter offpeak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average winter offpeak
The total system cost is = $ 268630.74700000003
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 0.00 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 0.00 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 0.00 
generator 16 = 1515.85 
generator 17 = 730.00 
generator 18 = 0.00 
generator 19 = 0.00 
generator 20 = 0.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 707.55 
line  2 = -62.40 
line  3 = -645.15 
LMPs in $/MWh
Node  1 = 24.00
Node  2 = 24.00
Node  3 = 24.00


#AOffPeakWinterDemand=df_AverloadsData.loc[:,'AWinterOffPeak'].to_numpy( )
APeakSummerDemand=df_AverloadsData.loc[:,'ASummerPeak'].to_numpy( )
AOffPeakSummerDemand=df_AverloadsData.loc[:,'ASummerOffPeak'].to_numpy( )
APeakFallDemand=df_AverloadsData.loc[:,'AFallPeak'].to_numpy( )
AOffPeakFallDemand=df_AverloadsData.loc[:,'AFallOffPeak'].to_numpy( )
APeakSpringDemand=df_AverloadsData.loc[:,'ASpringPeak'].to_numpy( )
AOffPeakSpringDemand=df_AverloadsData.loc[:,'ASpringOffPeak'].to_numpy( )

In [35]:
#UC MODEL of Average summer peak hour
def APeakSummerED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == APeakSummerDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [36]:
m= APeakSummerED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 575351.661
  Upper bound: 575351.661
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05520987510681152
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [37]:
print('SOLUTION-Average summer peak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average summer peak
The total system cost is = $ 575351.6610000001
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 2163.00 
generator  4 = 1530.50 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 734.45 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = -354.10 
line  2 = 900.00 
line  3 = -545.90 
LMPs in $/MWh
Node  1 = 37.50
Node  2 = 37.00
Node  3 = 38.00


In [38]:
#UC MODEL of Average summer offpeak hour
def AOffPeakSummerED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == AOffPeakSummerDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [39]:
m= AOffPeakSummerED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 379566.806
  Upper bound: 379566.806
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04039478302001953
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [40]:
print('SOLUTION-Average summer offpeak')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average summer offpeak
The total system cost is = $ 379566.8060000001
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 0.00 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 104.35 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 638.35 
line  2 = -172.95 
line  3 = -465.40 
LMPs in $/MWh
Node  1 = 33.00
Node  2 = 33.00
Node  3 = 33.00


In [41]:
#UC MODEL of Average fall peak hour
def APeakFallED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == APeakFallDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [42]:
m= APeakFallED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 395298.108
  Upper bound: 395298.108
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05657553672790527
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [43]:
print('SOLUTION-Average fall peak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average fall peak hour
The total system cost is = $ 395298.1080000001
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 292.15 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 634.83 
line  2 = -90.87 
line  3 = -543.97 
LMPs in $/MWh
Node  1 = 35.00
Node  2 = 35.00
Node  3 = 35.00


In [25]:
#UC MODEL of Average fall offpeak hour
def AOffPeakFallED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= PMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == AOffPeakFallDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [44]:
m= AOffPeakFallED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 324087.053
  Upper bound: 324087.053
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06478691101074219
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [45]:
print('SOLUTION-Average fall offpeak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average fall offpeak hour
The total system cost is = $ 324087.053
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 0.00 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 0.00 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 707.05 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 0.00 
generator 19 = 0.00 
generator 20 = 670.00 
generator 21 = 2337.70 
generator 22 = 2425.00 
generator 23 = 2716.00 
generator 24 = 2795.73 
generator 25 = 745.54 
generator 26 = 922.37 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 404.38 
line  2 = 122.73 
line  3 = -527.12 
LMPs in $/MWh
Node  1 = 30.00
Node  2 = 30.00
Node  3 = 30.00


In [59]:
#UC MODEL of Average spring peak hour
def APeakSpringED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= SpringPMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == APeakSpringDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m


In [60]:
m= APeakSpringED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 485668.9
  Upper bound: 485668.9
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04108905792236328
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [61]:
print('SOLUTION-Average spring peak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average spring peak hour
The total system cost is = $ 485668.9
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 1908.25 
generator  4 = 1530.50 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 753.70 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 799.20 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 1205.00 
generator 22 = 1250.00 
generator 23 = 1400.00 
generator 24 = 1441.10 
generator 25 = 384.30 
generator 26 = 475.45 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = -318.25 
line  2 = 900.00 
line  3 = -581.75 
LMPs in $/MWh
Node  1 = 37.50
Node  2 = 37.00
Node  3 = 38.00


In [62]:
#UC MODEL of Average spring peak hour
def APeakOffSpringED():
    m=ConcreteModel()
    #m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.N=Set(initialize=N)
    m.L=Set(initialize=L)
    m.G=Set(initialize=G)
    m.p=Var(m.G, bounds = (0,3500))#This is power generation. Could also declare as m.p=Var(m.G,within=PositiveReals)
    m.flow=Var(m.L, bounds = (-1000,1000))#This is power flow on a line
    m.system_cost=Objective(expr=sum(m.p[g]*VarCost[g] for g in m.G), sense=minimize)#Objective is to minimize costs
    m.MaxGeneration=Constraint(m.G, rule=lambda m, g:  m.p[g] <= SpringPMax[g])
    m.NodePowerBalanceConstraint=Constraint(m.N, rule=lambda m, n: sum(LineToBus[l,n]*m.flow[l] for l in L)-sum(LineFromBus[l,n]*m.flow[l] for l in L)+sum(GeneratorInBus[g,n]*m.p[g] for g in G) == AOffPeakSpringDemand[n])
    m.KVLAroundLoopConstraint=Constraint(rule=lambda m: sum(LineReactance[l]*m.flow[l] for l in L)==0)#Note that in the data we defined the lines L1-2, L2-3, and L3,1 . That way the coefficient of each one in the KVL loop is positive 1.
    m.MaxFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] <= LineCapacity[l])
    m.MaxCounterFlow=Constraint(m.L, rule=lambda m, l:  m.flow[l] >= -LineCapacity[l])
    return m

In [63]:
m= APeakOffSpringED()
SolverFactory('glpk',executable='D:\download\glpk-4.65\w64\glpsol.exe').solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 341557.1
  Upper bound: 341557.1
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 32
  Number of nonzeros: 72
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05663561820983887
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [64]:
print('SOLUTION-Average spring offpeak hour')
print('The total system cost is = $',m.system_cost())
print('Generation in MW')
for g in G:
        print("generator {0:2d} = {1:.2f} ".format(g+1, m.p[g]()))
print('Flow on transmission lines in MW')
for l in L:
        print("line {0:2d} = {1:.2f} ".format(l+1, m.flow[l]()))
print('LMPs in $/MWh')
for n in N:
        print("Node {0:2d} = {1:.2f}".format(n+1,m.dual[m.NodePowerBalanceConstraint[n]]))

SOLUTION-Average spring offpeak hour
The total system cost is = $ 341557.10000000003
Generation in MW
generator  1 = 0.00 
generator  2 = 0.00 
generator  3 = 0.00 
generator  4 = 1070.65 
generator  5 = 0.00 
generator  6 = 0.00 
generator  7 = 0.00 
generator  8 = 423.50 
generator  9 = 697.90 
generator 10 = 697.90 
generator 11 = 0.00 
generator 12 = 920.00 
generator 13 = 0.00 
generator 14 = 0.00 
generator 15 = 977.50 
generator 16 = 2244.80 
generator 17 = 730.00 
generator 18 = 271.20 
generator 19 = 979.70 
generator 20 = 670.00 
generator 21 = 1205.00 
generator 22 = 1250.00 
generator 23 = 1400.00 
generator 24 = 1441.10 
generator 25 = 384.30 
generator 26 = 475.45 
generator 27 = 0.00 
generator 28 = 0.00 
Flow on transmission lines in MW
line  1 = 410.77 
line  2 = 226.17 
line  3 = -636.93 
LMPs in $/MWh
Node  1 = 35.00
Node  2 = 35.00
Node  3 = 35.00
